In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from tqdm import tqdm
import urllib.parse

# ====== 使用者可自訂的參數 ======
base_url = "https://tabelog.com/tokyo/rstLst/"
page_count = 3                 # 要抓幾頁
rating_threshold = 3.5         # 評分篩選門檻
output_csv_name = "tabelog_高評分清單.csv"
# ==================================

headers = {"User-Agent": "Mozilla/5.0"}
df = pd.DataFrame(columns=["店名", "評分", "連結"])

# Step 1: 抓取多頁資料
for page in range(1, page_count + 1):
    print(f"📄 抓取第 {page} 頁...")
    url = f"{base_url}{page}/"
    try:
        res = requests.get(url, headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
    except Exception as e:
        print(f"⚠️ 第 {page} 頁錯誤：{e}")
        continue

    restaurants = soup.select(".list-rst__wrap")
    for item in restaurants:
        name_tag = item.select_one(".list-rst__rst-name-target")
        rating_tag = item.select_one(".c-rating__val")
        if name_tag and rating_tag:
            try:
                rating = float(rating_tag.text.strip())
            except ValueError:
                continue
            name = name_tag.text.strip()
            link = name_tag['href']
            df.loc[len(df)] = [name, rating, link]
    time.sleep(1)

print(f"\n✅ 共抓取 {len(df)} 筆資料。")

# Step 2: 篩選評分 > 門檻
df_high = df[df["評分"] > rating_threshold].copy()

# Step 3: 抓每家店的地址
df_high["地址"] = None
for idx, row in tqdm(df_high.iterrows(), total=len(df_high), desc="📍抓取地址中"):
    try:
        res = requests.get(row["連結"], headers=headers)
        soup = BeautifulSoup(res.text, "html.parser")
        addr_tag = soup.select_one(".rstinfo-table__address")
        address = addr_tag.text.strip() if addr_tag else "未找到"
    except Exception as e:
        address = f"錯誤：{e}"
    df_high.at[idx, "地址"] = address
    time.sleep(1)

# Step 4: 產生 Google Maps 搜尋網址（用地址）
df_high["Google地圖搜尋"] = df_high["地址"].apply(
    lambda addr: "https://www.google.com/maps/search/" + urllib.parse.quote(addr)
)

# Step 5: 匯出 CSV
df_high[["店名", "評分", "地址", "Google地圖搜尋"]].to_csv(
    output_csv_name, index=False, encoding="utf-8-sig"
)

print(f"\n✅ 已成功匯出：{output_csv_name}")


📄 抓取第 1 頁...
📄 抓取第 2 頁...
📄 抓取第 3 頁...

✅ 共抓取 60 筆資料。


📍抓取地址中: 100%|██████████████████████████████████████████████████████████████████████| 6/6 [00:11<00:00,  1.92s/it]


✅ 已成功匯出：tabelog_高評分清單.csv
